# AutoGen RoundRobinGroupChat
A team is a group of agents that work together to achieve a common goal.


## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

azure_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating a Team

RoundRobinGroupChat is a team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We create a team with two AssistantAgent and a TextMentionTermination condition that stops the team when a specific word is detected in the agent’s response.

In [ ]:
import asyncio
import aiofiles
import arxiv
import json
import os

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

PAPER_DIR = "../ArxivResearcher/papers"

# Create the user proxy agent.
user_proxy_agent = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.


# Use arxiv to find the papers 
async def search_arxiv(topic, max_results=5):
    client = arxiv.Client()

    # Search for the latest articles matching the queried topic
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = await client.results(search)

    return list(papers)

# Create the arxiv search agent.
arxiv_search_agent = AssistantAgent(
    name="arxiv_search",
    description="An agent that searches for research papers related to topic on arXiv.",
    model_client=azure_model_client,
    tools=[search_arxiv],
    system_message="You are an expert academic search engine. \
        Your task is to find the most relevant and recent papers on arXiv \
        for a given topic and returning the results in python list."
)

async def extract_content(topic, papers):
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")
    # Try to load existing papers info
    try:
        async with aiofiles.open(file_path, "r") as json_file:
            contents = await json_file.read()
            papers_info = json.loads(contents)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    async with aiofiles.open(file_path, "w") as json_file:
        await json_file.write(json.dumps(papers_info, indent=2))

    return f"Saved paper metadata to: {file_path}. Paper IDs: {paper_ids}"

# Create the Content Extractor agent.
info_extractor_agent = AssistantAgent(
    name="content_extractor",
    description="An agent that extracts relevant information from research papers.",
    model_client=azure_model_client,
    tools=[extract_content],
    system_message="You are an expert in extracting relevant information from research papers. \
        Given a topic and a list of arxiv research papers, extract the title, authors, summary, pdf_url, and published date of each paper. \
        The file should be saved in a JSON file in a directory named after the topic, with spaces replaced by underscores. \
        The JSON file should be named 'papers_info.json'."
)
# Create the Research Summarizer agent.
research_summarizer_agent = AssistantAgent(
    name="research_summarizer",
    description="An agent that creates a concise summary on a topic from the given research papers.",
    model_client=azure_model_client,
    system_message="You are an expert academic researcher. \
        The JSON file 'papers_info.json' will be located at {PAPER_DIR} and inside a subdirectory named after the topic, with spaces replaced by underscores. \
        The file contains the titles, authors, summaries, pdf_urls, and published dates of the papers. \
        Your task is to synthesize the main findings and contributions from multiple research paper \
        abstracts into a single, concise, and coherent overview of the topic. \
        Focus on common themes and significant advancements."    
)

# Define a termination condition that stops the task
text_mention_termination = TextMentionTermination("QUIT", "STOP", "EXIT", "END", "CANCEL")
max_message_termination = MaxMessageTermination(max_message=20)
termination = text_mention_termination | max_message_termination

# Create a team with the arxiv search and context extractor agents.
team = RoundRobinGroupChat(
    [user_proxy_agent, arxiv_search_agent, info_extractor_agent, research_summarizer_agent],
    termination_condition=termination
)

KeyError: 'topic'

## Running a Team
Call the run() method to start the team with a task.

The termination condition was met when the word “APPROVE” is detected in the agent’s response. When the team stops, it returns a TaskResult object with all the messages produced by the agents in the team.

In [ ]:
# Stream the messages to the console.
await Console(team.run_stream(task="model context protocol"))  
#await team.reset()  # Reset the team for the next run.

messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 49, 59, 378389, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=126), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 50, 2, 62631, tzinfo=datetime.timezone.utc), content="In autumn's embrace, the leaves ignite,  \nA tapestry of amber, gold, and bright.  \nWhispers of wind through the branches sway,  \nAs twilight descends, marking shorter days.  \n\nPumpkins abound in fields aglow,  \nWhile harvest moon casts a gentle show.  \nCrisp air carries scents of woodsmoke and spice,  \nNature's farewell, so tenderly nice.  \n\nThe world transforms in a vibrant parade,  \nWhere memories linger, and colors invade.  \nIn the fall's soft dance, we find our grace,  \nAs seasons shift, and time leaves its trace.  ", type='TextMe

## Stopping a Team
You can also stop the team from outside by using the ExternalTermination.

Calling set() on ExternalTermination will stop the team when the current agent’s turn is over. 

Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.

In [1]:
'''
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination, TokenUsageTermination, TimeoutTermination, SourceMatchTermination, HandoffTermination, StopMessageTermination

# MaxMessageTermination
termination = MaxMessageTermination(5) # terminate after 5 messages
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)

await Console(team.run_stream(task="Give a number"))

#TextMentionTermination
termination = TextMentionTermination("quit") # Terminate when the message contains "quit".
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task=""))

#TokenUsageTermination
termination = TokenUsageTermination(100) # terminate when token usage is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))'''

'\nfrom autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination, TokenUsageTermination, TimeoutTermination, SourceMatchTermination, HandoffTermination, StopMessageTermination\n\n# MaxMessageTermination\ntermination = MaxMessageTermination(5) # terminate after 5 messages\nteam = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)\n\nawait Console(team.run_stream(task="Give a number"))\n\n#TextMentionTermination\ntermination = TextMentionTermination("quit") # Terminate when the message contains "quit".\nteam = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)\nawait Console(team.run_stream(task=""))\n\n#TokenUsageTermination\ntermination = TokenUsageTermination(100) # terminate when token usage is met.\nteam = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)\nawait Console(team.run_stream(task="Give a nu

## Aborting a Team
You can abort a call to run() or run_stream() during execution by setting a CancellationToken passed to the cancellation_token parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [ ]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="model context protocol.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.
